# CE + CL VIT-TINY-Patch SIZE 2

In [2]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from transformers import (
    ViTConfig,
    ViTPreTrainedModel,
    ViTModel,
    get_cosine_schedule_with_warmup
)
from tqdm.auto import tqdm
import random

# Setup & Seeding for reproducibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_dp = torch.cuda.device_count() > 1
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

# Initial transform to split dataset
init_tf = transforms.ToTensor()
dataset_full = datasets.CIFAR10('./data', train=True, download=True, transform=init_tf)
val_size = int(0.1 * len(dataset_full))
train_size = len(dataset_full) - val_size
train_ds, val_ds = torch.utils.data.random_split(
    dataset_full,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

batch_size = 128

# Precompute DataLoaders for each resolution
stages = [(r) for r in [(12, 10), (16, 10), (20, 10), (24, 10), (28, 10), (32, 10)]]
dataloader_dict = {}

for resolution, _ in stages:
    train_tf = transforms.Compose([
        transforms.RandomCrop(resolution, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    val_tf = transforms.Compose([
        transforms.Resize(resolution),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    train_set = datasets.CIFAR10('./data', train=True, download=False, transform=train_tf)
    val_set   = datasets.CIFAR10('./data', train=True, download=False, transform=val_tf)

    train_loader = DataLoader(Subset(train_set, train_ds.indices), batch_size=batch_size,
                              shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(Subset(val_set, val_ds.indices), batch_size=batch_size,
                            shuffle=False, num_workers=0, pin_memory=True)

    dataloader_dict[resolution] = {
        'train': train_loader,
        'val': val_loader
    }

# Test loader (fixed resolution)
test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
test_ds = datasets.CIFAR10('./data', train=False, transform=test_tf)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# # ViT-Tiny Classifier
class ViTTinyClassifier(ViTPreTrainedModel):
    def __init__(self, config: ViTConfig):
        super().__init__(config)
        self.vit = ViTModel(config)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(self, pixel_values, labels=None):
        outputs = self.vit(
            pixel_values,
            interpolate_pos_encoding=True,
            return_dict=True
        )
        cls_output = outputs.last_hidden_state[:, 0]
        logits = self.classifier(cls_output)
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)
        return {"logits": logits, "loss": loss}

# # Instantiate model
tiny_config = ViTConfig(
    hidden_size=192,
    num_hidden_layers=12,
    num_attention_heads=3,
    intermediate_size=768,
    patch_size=2,
    image_size=32,
    num_labels=10,
    hidden_dropout_prob=0.1,
    classifier_dropout_prob=0.1
)
student = ViTTinyClassifier(tiny_config).to(device)
student = nn.DataParallel(student)

no_decay = ["bias", "LayerNorm.weight"]
decay_params = [p for n,p in student.named_parameters() if not any(nd in n for nd in no_decay)]
nodecay_params = [p for n,p in student.named_parameters() if     any(nd in n for nd in no_decay)]

optimizer = optim.AdamW([
    {"params": decay_params,   "weight_decay": 1e-4},
    {"params": nodecay_params, "weight_decay": 0.0},
], lr=3e-4)


# Evaluation function
def eval_model(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(pixel_values=imgs)
            preds = outputs["logits"].argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total

# Curriculum Training
for resolution, epochs in stages:
    print(f"\n=== Training at resolution {resolution}px ===")
    tr_loader = dataloader_dict[resolution]['train']
    vl_loader = dataloader_dict[resolution]['val']

    best_val_acc = 0
    no_improve = 0
    patience = 3

    for epoch in range(1, epochs + 1):
        student.train()
        train_correct = train_total = 0
        for imgs, labels in tqdm(tr_loader, desc=f"Epoch {epoch}/{epochs}"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = student(pixel_values=imgs, labels=labels,)
            loss = outputs["loss"]
            if loss is not None and loss.dim() > 0:
                loss = loss.mean()
            loss.backward()
            optimizer.step()

            preds = outputs["logits"].argmax(1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        train_acc = 100 * train_correct / train_total
        val_acc = eval_model(student, vl_loader)
        print(f"{resolution}px Ep{epoch}: Train {train_acc:.2f}%  Val {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve = 0
            torch.save(student.state_dict(), f'best_tiny_{resolution}px.pth')
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"→ Early stopping at resolution {resolution}px")
                student.load_state_dict(torch.load(f'best_tiny_{resolution}px.pth'))
                break

    print(f"=> Stage {resolution}px best Val = {best_val_acc:.2f}%")

# Final test evaluation
student.load_state_dict(torch.load('best_tiny_32px.pth'))
final_test_acc = eval_model(student, test_loader)
print(f"\n🌟 Final ViT-Tiny Test Acc: {final_test_acc:.2f}%")

Files already downloaded and verified

=== Training at resolution 12px ===


Epoch 1/10:   0%|          | 0/352 [00:00<?, ?it/s]

12px Ep1: Train 20.76%  Val 26.08%


Epoch 2/10:   0%|          | 0/352 [00:00<?, ?it/s]

12px Ep2: Train 24.38%  Val 30.32%


...

32px Ep9: Train 76.63%  Val 71.88%


Epoch 10/10:   0%|          | 0/352 [00:00<?, ?it/s]

32px Ep10: Train 76.82%  Val 72.54%
=> Stage 32px best Val = 72.54%


/tmp/ipykernel_31/2348555776.py:184: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load('best_tiny_32px.pth'))



🌟 Final ViT-Tiny Test Acc: 71.11%
